# IS5152 Project
## Beijing PM2.5 prediction

In [1]:
import os

# data manipulation
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

# visualiation
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

from utils import read_data

# Load Data

In [2]:
count, df = read_data()
df.head()

Loaded  data/PRSA_Data_Gucheng_20130301-20170228.csv
Loaded  data/PRSA_Data_Huairou_20130301-20170228.csv
Loaded  data/PRSA_Data_Tiantan_20130301-20170228.csv
Loaded  data/PRSA_Data_Changping_20130301-20170228.csv
Loaded  data/PRSA_Data_Guanyuan_20130301-20170228.csv
Loaded  data/PRSA_Data_Nongzhanguan_20130301-20170228.csv
Loaded  data/PRSA_Data_Wanliu_20130301-20170228.csv
Loaded  data/PRSA_Data_Dongsi_20130301-20170228.csv
Loaded  data/PRSA_Data_Wanshouxigong_20130301-20170228.csv
Loaded  data/PRSA_Data_Aotizhongxin_20130301-20170228.csv
Loaded  data/PRSA_Data_Dingling_20130301-20170228.csv
Loaded  data/PRSA_Data_Shunyi_20130301-20170228.csv
Applied normalization on  ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
35064 rows per station, total 382168 rows


,time_stamp,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM,station,WD_E,WD_ENE,WD_ESE,WD_N,WD_NE,WD_NNE,WD_NNW,WD_NW,WD_S,WD_SE,WD_SSE,WD_SSW,WD_SW,WD_W,WD_WNW,WD_WSW
20,21,13.0,0.023069,0.023442,0.010417,0.101010,0.056767,0.349593,0.740066,0.270353,0.0,0.143939,Gucheng,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
21,22,15.0,0.021063,0.027444,0.038194,0.111111,0.048362,0.339837,0.756623,0.285714,0.0,0.053030,Gucheng,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
22,23,16.0,0.026078,0.031447,0.059028,0.111111,0.041825,0.344715,0.761589,0.282642,0.0,0.075758,Gucheng,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,24,16.0,0.026078,0.027444,0.062500,0.101010,0.040891,0.326829,0.764901,0.282642,0.0,0.083333,Gucheng,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
24,25,14.0,0.016048,0.047456,0.142361,0.131212,0.023147,0.318699,0.769868,0.276498,0.0,0.106061,Gucheng,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Split data:

In [9]:
## Shuffle df
import random

## Reindex from 0, original index can be used to split time series
if 'original_index' not in df.columns:
    df = df.reset_index()
    df = df.rename(columns={"index": "original_index"})

df.head()

def shuffle(df):
    index = list(df.index)
    random.shuffle(index)
    df = df.iloc[index]
    return df

df = shuffle(df)
df.head(10)

,original_index,time_stamp,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM,station,WD_E,WD_ENE,WD_ESE,WD_N,WD_NE,WD_NNE,WD_NNW,WD_NW,WD_S,WD_SE,WD_SSE,WD_SSW,WD_SW,WD_W,WD_WNW,WD_WSW
200398,217553,7170,334.0,0.339017,0.207547,0.527778,0.525253,0.000000,0.307317,0.713576,0.433180,0.000000,0.098485,Wanliu,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
65330,71390,1263,222.0,0.177533,0.195540,0.135417,0.191919,0.047429,0.570732,0.438742,0.652842,0.000000,0.166667,Tiantan,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
287587,314183,33672,403.0,0.468405,0.009434,0.475694,0.606061,0.005403,0.245528,0.642384,0.457757,0.000000,0.083333,Wanshouxigong,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34830,38159,3096,141.0,0.171515,0.009434,0.013889,0.101010,0.161364,0.700813,0.167219,0.909370,0.227586,0.106061,Huairou,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
276823,303007,22496,66.0,0.084253,0.003431,0.260417,0.101010,0.001668,0.624390,0.397351,0.800307,0.000000,0.121212,Wanshouxigong,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
61604,67314,32251,323.0,0.441324,0.009434,0.461806,0.262626,0.001668,0.443902,0.390728,0.617512,0.000000,0.075758,Huairou,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
207704,225839,15456,12.0,0.038114,0.027444,0.222222,0.070707,0.001668,0.268293,0.804636,0.196621,0.000000,0.090909,Wanliu,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
158313,171461,31206,52.0,0.106319,0.003431,0.312500,0.070707,0.001668,0.593496,0.539735,0.754224,0.000000,0.098485,Guanyuan,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
129308,139812,34621,12.0,0.033099,0.011435,0.048611,0.030303,0.044627,0.344715,0.642384,0.228879,0.000000,0.477273,Changping,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
40179,44122,9059,31.0,0.044132,0.041452,0.086806,0.080808,0.058635,0.465041,0.523179,0.271889,0.000000,0.075758,Huairou,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [4]:
# split data into trainng 90% and test 10%
# split training data into trsining set and validation set 80:20
total_data_count = len(df)
test_data_count = int(total_data_count*0.1)
training_data_count = int((total_data_count - test_data_count)*0.8)

test_df = df[0: test_data_count]
training_df = df[test_data_count : training_data_count + test_data_count]
validation_df = df[training_data_count + test_data_count ::]
print("train: %d, val: %d, test: %d" % (len(test_df),len(training_df),len(validation_df)))


train: 38216, val: 275161, test: 68791


# Simple Linear Regression:

In [5]:
# import torch
# import torch.nn as nn
# import numpy as np
# from torch.autograd import Variable
# test_local = training_df #test using CPU only, too slow to process 25k data

# target_col = test_local["PM2.5"].to_numpy(dtype=np.float32,copy=True).reshape(-1,1)
# data_col = test_local.drop(columns=["PM2.5"]).to_numpy(dtype=np.float32,copy=True)

# val_y = validation_df["PM2.5"].to_numpy(dtype=np.float32,copy=True).reshape(-1,1)
# val_x = validation_df.drop(columns=["PM2.5"]).to_numpy(dtype=np.float32,copy=True)


# print("Training Data",data_col.shape,target_col.shape)
# print("Validation Data",val_x.shape,val_y.shape)


# model = torch.nn.Sequential(
#     torch.nn.Linear(8, 1),
# )

# enable_cuda = True

# if torch.cuda.is_available() and enable_cuda:
#     print("Cuda is available!")
#     model.cuda()
# else:
#     print("Cuda not available, using CPU")

# loss_fn = nn.MSELoss()
# # loss_fn = nn.L1Loss()
# # loss_fn = nn.SmoothL1Loss()
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-05)
# # optimizer = torch.optim.SGD(model.parameters(),lr=1e-05)

# train_loss_history = []
# val_loss_history = []

# for epoch in range(300000):
    
#     ## Training
# #     model.train(True)
#     if torch.cuda.is_available() and enable_cuda:
#         y = Variable(torch.from_numpy(target_col).cuda())
#         x = Variable(torch.from_numpy(data_col).cuda())
#     else:
#         y = Variable(torch.from_numpy(target_col))
#         x = Variable(torch.from_numpy(data_col))
#     y_pred = model(x)
    
#     training_loss = loss_fn(y_pred, y)
    
#     optimizer.zero_grad()
#     training_loss.backward()
#     optimizer.step()
    
#      ## Validation
# #     model.train(False)
#     if torch.cuda.is_available() and enable_cuda:
#         y = Variable(torch.from_numpy(val_y).cuda())
#         x = Variable(torch.from_numpy(val_x).cuda())
#     else:
#         y = Variable(torch.from_numpy(val_y))
#         x = Variable(torch.from_numpy(val_x))
#     y_pred = model(x)
#     val_loss = loss_fn(y_pred,y)
#     train_loss_history.append(training_loss.item())
#     val_loss_history.append(val_loss.item())

    
#     if epoch % 1000 == 0:
#         print('Epoch {}, training loss {}, val loss {}'.format(epoch,training_loss.item(),val_loss.item()))
       
        

# import matplotlib.pyplot as plt

# %matplotlib inline

# # Plot training & validation accuracy values
# plt.plot(train_loss_history)
# plt.plot(val_loss_history)
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

In [6]:
# ## Test
# def test(test_y, test_x):
#     if torch.cuda.is_available() and enable_cuda:
#         y = Variable(torch.from_numpy(test_y).cuda())
#         x = Variable(torch.from_numpy(test_x).cuda())
#     else:
#         y = Variable(torch.from_numpy(test_y))
#         x = Variable(torch.from_numpy(test_x))
#     y_pred = model(x)
#     test_loss = loss_fn(y_pred,y)
#     return y_pred, test_loss.item()

# test_y = test_df["PM2.5"].to_numpy(dtype=np.float32,copy=True).reshape(-1,1)
# test_x = test_df.drop(columns=["PM2.5"]).to_numpy(dtype=np.float32,copy=True)
# y_pred, test_loss = test(test_y, test_x)

# result = test_df.copy()
# result['Predict_PM2.5'] = y_pred.type(torch.FloatTensor).cpu().detach().numpy().astype(float)

# print(result.iloc[1])
# print('Test loss is {}'.format(test_loss))

In [7]:
# model.state_dict()

In [8]:
# df.head()